# Домашнее задание по теме «Нейронные сети»

Сегодня ты продолжишь решать задачу классификации чисел MNIST с помощью нейросети.

В этом домашнем задании тебе нужно:
- добавить вычисление метрики на этапе валидации;
- вспомнить про нормализацию данных и применить её;
- познакомиться с популярным блоком нейросети `BatchNorm`;
- применить шедулер.

## Задача 1. Метрика на валидации [3 балла]

На семинаре во время валидации нейросети мы считали `loss`. Это вполне корректное действие, и так можно и нужно валидироваться. Однако `loss` далеко не всегда понятен для бизнеса как мера качества решения задачи. Для этого существуют метрики.

Проблема в том, что не все метрики можно оптимизировать напрямую. Но если нужно получить оценку качества работы нашего алгоритма именно на метрике, то вычислить её можно на этапе валидации.

В этом и состоит первая задача. Для этого нужно:
1. Добавить вычисление метрики `Accuracy` из `sklearn` после этапа валидации на эпохе (подробности в коде). **[1 балл]**
2. Добавить в `print` лог, который выводит значение `Accuracy` на валидации. **[0,5 балла]**
3. Обучить модель с семинара с реализованным вычислением `Accuracy` **[0,5 балла по метрике]**. Не забудь скачать веса лучшей модели и прикрепить их к домашнему заданию.
4. Описать, что получилось. Обрати особое внимание, как соотносится `loss` и значение метрики на валидации. Всегда ли оптимальная модель по `loss` оптимальна и по метрике? **[1 балл]**

> **Важно.** Запомни, какой результат получился на валидации. Дальше ты будешь усложнять решение задачи. Это повлияет на качество её решения.

### Твоё решение задачи 1

> **Подсказка.** Следующие несколько ячеек — код с семинара. Просто запусти их. Когда тебе потребуется что-то написать, это будет выделено жирным.

In [30]:
# Импортируем нужные пакеты
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST

from tqdm import tqdm

In [31]:
# Загружаем датасет
train_ = MNIST('../Datasets', # Папка для сохранения или загрузки
              download=True, # Если нет в папке, скачиваем из интернета
              train=True,
              ) # train-подвыборка
test = MNIST("../Datasets", download=True, train=False)

# Для начала достанем все данные из нашего датасета
X_train, y_train = train_.data, train_.targets
X_test, y_test = test.data, test.targets

In [32]:
# Определим класс датасета
class DatasetMNIST(Dataset):

  def __init__(self, X, y, device="cuda"):
    self.X = X.flatten(start_dim=1) / 255
    self.y = y
    self.device=device
  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.X[idx].to(self.device), self.y[idx].to(self.device)

In [33]:
# Определим новые датасеты
train_ds = DatasetMNIST(X_train, y_train)
test_ds = DatasetMNIST(X_test, y_test)

In [34]:
# Определим DataLoader
train_dl = DataLoader(
    train_ds, # Наш датасет
    batch_size=64, # Размер батча. Меньше 32, согласно многим исследованиям, ставить не рекоммендуется из-за потерь в качестве
    shuffle=True, # Указываем, перемешивать ли данные перед каждой эпохой (проходом по данным). Для train-подвыборки всегда ставим True, кроме единичных исключений
    drop_last=True, # Если наш последний батч будет неполным, то не обучаемся на нём
    num_workers=4, # Указываем, сколько процессов будут собирать данные в батч. Обычно выбирают по числу ядер
    persistent_workers=True # Используем, чтобы не создавать каждый раз новый процесс при обращении к DataLoader. Полезно для небольшого ускорения исполнения
)

test_dl = DataLoader(
    test_ds, # Тестовый датасет
    batch_size=64*4, # Для скорости можно установить значение больше, чем на train. Если только получаем предсказания, а не обучаемся, то нужно меньше ресурсов, а значит, в GPU поместится батч большего размера
    shuffle=False, # Не будем перемешивать
    drop_last=False, # И исключать неполный батч тоже не будем, потому что нам нужны предсказания для него
    num_workers=4,
    persistent_workers=True
)

In [35]:
# Функция для одного шага обучения. Вставь код с семинара
def train_step(batch, model, loss, optimizer, device):

    # Обнуляем градиенты
    model.zero_grad()

    X, y = batch
    # Раздели батч на данные и метку = batch
    X = X.to(device)
    y = y.to(device)

    # Пропускаем данные через модель
    # Логиты — выход из последнего слоя нейросети, но основе которых решается задача
    logits = model(X)
    # Считаем loss
    l = loss(logits, y)

    # Обратное распространение ошибки
    l.backward()

    # Шаг оптимизатора
    optimizer.step()

    return l.item()

In [36]:
# Функция для одного шага обучения
def train_step(batch, model, loss, optimizer, device):

    X, y = batch
    X = X.to(device)
    y = y.to(device)

    model.zero_grad()

    logits = model(X)
    l = loss(logits, y)

    l.backward()
    optimizer.step()

    return l.item()

In [37]:
# Функция для обучения на эпохе
def train(model, loss, optimizer, device, train_dataloader):
    model.train()
    train_loss = 0

    for batch in tqdm(train_dataloader):
      loss_step = train_step(batch, model, loss, optimizer, device)
      train_loss += loss_step / len(train_dataloader)

    return train_loss

In [38]:
# Функция для одного шага валидации
def valid_step(batch, model, loss, device):

      X, y = batch
      X = X.to(device)
      y = y.to(device)

      with torch.no_grad():
        logits = model(X)
        l = loss(logits, y)

      return logits.argmax(dim=-1).detach().cpu().numpy(), l.item()

In [39]:
# Функция для всей валидации на эпохе, будем использовать её также для получения предсказаний
def validate(model, loss, device, val_dataloader):
  model.eval()
  val_loss = 0
  preds = []
  for batch in tqdm(val_dataloader):
    preds_step, loss_step = valid_step(batch, model, loss, device)

    val_loss += loss_step / len(val_dataloader)
    preds.append(preds_step)

  preds = np.concatenate(preds)

  return preds, val_loss

Добавь в функцию `train_and_validate`:
* вычисление `Accuracy` после валидации;
* печать соответствующего лога.

> **Добавь свой код в ячейки ниже.**

In [40]:
def train_and_validate(
    epochs,
    model,
    loss,
    optimizer,
    device,
    train_dataloader,
    val_dataloader,
    save_every=1,
    naming="",
):

    model.to(device)
    best_acc = -1
    for e in range(epochs):

        train_loss = train(model, loss, optimizer, device, train_dataloader)
        val_preds, val_loss = validate(model, loss, device, val_dataloader)

        val_targets = torch.cat([y.to(device) for _, y in val_dataloader])
        valid_acc = (val_preds == val_targets.cpu()).numpy().mean()

        print(
            f"Эпоха: {e} | Train Loss {train_loss} | Val Loss {val_loss} | Val_acc {valid_acc}"
        )  # Добавь в print полученное значение с соответствующей подписью

        if e % save_every == 0 and valid_acc > best_acc:
            torch.save(
                model.state_dict(), f"model_epoch_{e}{naming}_acc_{valid_acc:.4f}.pth"
            )
            best_acc = valid_acc

In [41]:
# Определяем модель с семинара, тут можешь ничего не трогать. Сеть с семинара подойдёт
class FCMNIST(nn.Module):

  def __init__(self):
    super().__init__() # Не забываем super init сделать, без этого ничего работать не будет

    # Линейный слой —> ReLU —> Линейный слой —> и так далее
    self.net = nn.Sequential(
        nn.Linear(784, 392),
        nn.ReLU(),
        nn.Linear(392, 191),
        nn.ReLU(),
        nn.Linear(191, 80),
        nn.ReLU(),
        nn.Linear(80, 40),
        nn.ReLU(),
        nn.Linear(40, 10)
    )

  def forward(self, X):
    return self.net(X)

In [42]:
model = FCMNIST()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
loss = (
    nn.CrossEntropyLoss()
)  # Кросс-энтропия, самая популярная функция потерь для решения задачи классификации. Разбиралась на лекциях
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
# Запусти
train_and_validate(epochs, model, loss, optimizer, device, train_dl, test_dl)

  0%|          | 0/937 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/rlohaw/Homeworks/AI/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/rlohaw/Homeworks/AI/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/rlohaw/Homeworks/AI/.venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_150254/2139042691.py", line 12, in __getitem__
    return self.X[idx].to(self.device), self.y[idx].to(self.device)
  File "/home/rlohaw/Homeworks/AI/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py", line 305, in _lazy_init
    raise RuntimeError(
RuntimeError: Cannot re-initiali

> **Важно.** Не забудь скачать веса лучшей модели!

Какие результаты у тебя получились? Является ли лучшая модель по loss также лучшей и по метрике?

## Задача 2. Нормализация данных [4 балла]

Мы уже нормализовали данные, чтобы улучшить качество решения линейных моделей. Для нейросетей это также хорошая практика.

В этой задаче тебе предстоит сделать нормализацию выходных данных, а именно:
1. Модифицировать класс датасета так, чтобы он нормализовал данные для обучения и выводил статистики train-подвыборки. **[2 балла]**
2. Инициализировать класс с полученными статистиками для валидационной подвыборки. **[1 балл]**
3. Обучить нейросеть и вывести все логи. **[0,5 балла]**
4. Описать, что получилось. Стало ли лучше? **[0,5 балла]**

Нормализация — это стандартизация наших данных. То есть мы вычитаем из нашей выборки среднее переменных по всей выборке и делим результат на стандартное отклонение выборки для каждой переменной:

$$X_{norm} = \frac{X - E(X)}{\sqrt{V(X)}}.$$

Также важно учесть, что, хоть у нас и векторы, мы работаем с картинками. Их нормализуют по каналам. Всего их обычно 3 (RGB — Red, Green, Blue). Но в нашем простом датасете канал всего один.

То есть нужно получить всего одно число для среднего и стандартного отклонений, а не 784, если бы нормализовали данные как табличные.

### Твоё решение задания 2

In [ ]:
class DatasetMNIST(Dataset):

    def __init__(
        self, X, y, mean=None, std=None, device="cuda"
    ):  # Теперь можно передавать статистики в датасет

        # Тут всё как было
        self.X = X.flatten(1) / 255
        self.y = y
        self.device = device # кажется кто-то забыл при создании задачи отправить датасетик на гпу)))))))

        # Если у нас нет среднего или стандартного отклонения
        if not mean or not std:
            mean = self.X.mean()  # Посчитай среднее
            std = self.X.std()  # Посчитай стандартное отклонение
            print(
                mean, std
            )  # print, потому что init в Dataset не должен ничего возвращать по правилам PyTorch

        self.X = (self.X - mean) / std

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].to(device), self.y[idx].to(device)

In [ ]:
# Надо пересоздать датасеты и даталоадеры
# Определим train-датасет
train_ds = DatasetMNIST(X_train, y_train)
train_mean = train_ds.X.mean()
train_std = train_ds.X.std()

tensor(0.1307) tensor(0.3081)


In [ ]:
# Передай полученные статистики в наш валидационный датасет
test_ds = DatasetMNIST(X_test, y_test, train_mean, train_std)

In [ ]:
# Определим даталоадеры
train_dl = DataLoader(
    train_ds, # Наш датасет
    batch_size=64, # Размер батча. Меньше 32, согласно многим исследованиям, ставить не рекомендуется из-за потерь в качетсве
    shuffle=True, # Указываем, перемешивать ли данные перед каждой эпохой (проходом по данным). Для train-подвыборки всегда ставим True, кроме единичных исключений
    drop_last=True, # Если наш последний батч будет неполным, то не обучаемся на нём
    num_workers=2, # Показывает, сколько процессов будет собирать данные в батч. Обычно выбирают по числу ядер
    persistent_workers=True # Указываем, чтобы не создавать каждый раз новый процесс при обращении к DataLoader. Полезно для небольшого ускорения исполнения
)

test_dl = DataLoader(
    test_ds, # Тестовый датасет
    batch_size=64*4, # Для скорости можно побольше поставить, чем на train. Так как только получаем предсказания, а не обучаемся, нужно меньше ресурсов, а значит, в GPU поместится батч большего размера
    shuffle=False, # Не будем перемешивать
    drop_last=False, # И исключать неполный батч не будем, потому что нам нужны предсказания для него
    num_workers=2,
    persistent_workers=True
)

In [ ]:
# Инициализируем модель
model = FCMNIST()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
loss = nn.CrossEntropyLoss() # Кросс-энтропия, самая популярная функция потерь для решения задачи классификации. Разбиралась на лекциях
epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Запусти обучение
train_and_validate(epochs, model, loss, optimizer, device, train_dl, test_dl, naming='_norm') # Здесь еще указали дополнительный нейминг модели, чтобы отличать веса

100%|██████████| 40/40 [00:00<00:00, 89.57it/s]


Эпоха: 0 | Train Loss 0.48496546616763764 | Val Loss 0.7366105735301971 | Val_acc 0.9354


100%|██████████| 40/40 [00:00<00:00, 77.41it/s]


Эпоха: 1 | Train Loss 0.11145784535118264 | Val Loss 0.5930806756019593 | Val_acc 0.9274


100%|██████████| 40/40 [00:00<00:00, 94.07it/s]


...

Эпоха: 7 | Train Loss 0.017989676953672953 | Val Loss 0.27109190318733445 | Val_acc 0.9439


100%|██████████| 40/40 [00:00<00:00, 106.97it/s]


Эпоха: 8 | Train Loss 0.012826820606942426 | Val Loss 0.21116707855835565 | Val_acc 0.9512


100%|██████████| 40/40 [00:00<00:00, 62.66it/s]


Эпоха: 9 | Train Loss 0.009580554261098646 | Val Loss 0.16368222180753947 | Val_acc 0.9661


> **Важно.** Не забудь сохранить веса лучшей модели на валидации!

Удалось ли улучшить качество? Должно получиться где-то +0,1–0,3 п. п. к решению без нормализации. Если не вышло, не значит, что ты сделал что-то неправильно. Датасет настолько простой, что задача хорошо решается и без нормализации данных.

## Задача 3. BatchNorm [3 балла]

Мы нормируем только входные данные. Но после каждого блока нейросети на выходе получаем новое представление наших данных. И нет никаких гарантий, что оно нормализовано, поэтому в нейросетях часто используют блок [BatchNorm](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html).

Он нормализует наши данные, как мы это уже сделали вручную для входа. Зачем BatchNorm тогда нужен? Мы производим обучение по батчам. Удобно считать необходимые для нормализации среднее арифметическое и отклонение не по всей выборке, а у каждого батча, постепенно накапливая эти значения. Именно этим занимается BatchNorm!

Также у этого слоя есть параметры: коэффициент умножения и сдвиг. Это нужно для того, чтобы слой мог при необходимости «отменить» или модифицировать эффект нормализации, унмножив или добавив определённое число.

В этой задаче тебе предстоит поработать над архитектурой нейросети, добавив BatchNorm между слоями.

BatchNorm может быть разным, для этой задачи используй [BatchNorm1d](https://www.google.com/url?q=https%3A%2F%2Fpytorch.org%2Fdocs%2Fstable%2Fgenerated%2Ftorch.nn.BatchNorm1d.html).

Тебе нужно:
1. Модифицировать нейросеть, добавив BatchNorm. **[1 балл]**

Слой BatchNorm можно ставить как до, так и после функции активации. Можешь попробовать разные варианты и посмотреть, какой будет лучше!

Этот слой принимает на вход один аргумент — размер входных данных. Поэтому, если на вход BatchNorm подаётся вектор длиной 100, надо инициализировать её через `nn.BatchNorm1d(100)`.
2. Обучить все и сохранить лучшую модель. **[1 балла]**
3. Описать, что получилось. **[1 балла]**

### Твоё решение задания 3

In [ ]:
class NormFCMNIST(nn.Module): # Norm добавлено в название, чтобы различать классы

  def __init__(self):
    super().__init__() # Не забываем super init сделать, без этого ничего работать не будет
    # Не надо вставлять BatchNorm до и после активации. Выбери одно расположение
    # Линейный слой —> (?Batchnorm) —> ReLU —> (?Batchnorm) —> Линейный слой —> и так далее
    self.net = nn.Sequential(
      
        nn.Linear(784, 392),
        nn.BatchNorm1d(392),
        nn.ReLU(),
        
        nn.Linear(392, 191),
        nn.BatchNorm1d(191),
        nn.ReLU(),

        nn.Linear(191, 80),
        nn.BatchNorm1d(80),
        nn.ReLU(),

        nn.Linear(80, 40),
        nn.BatchNorm1d(40),
        nn.ReLU(),

        nn.Linear(40, 10)
    )

  def forward(self, X):
    return self.net(X)

Объясни, куда ты добавил BatchNorm и почему?

In [ ]:
# Инициализируем модель
model = NormFCMNIST()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
loss = nn.CrossEntropyLoss() # Кросс-энтропия, самая популярная функция потерь для решения задачи классификации. Разбиралась на лекциях
epochs = 30
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Запусти обучение
train_and_validate(epochs, model, loss, optimizer, device, train_dl, test_dl, naming='_batchnorm') # Нейминг опять другой

100%|██████████| 40/40 [00:00<00:00, 94.61it/s]


Эпоха: 0 | Train Loss 0.2275548795428976 | Val Loss 1.9261489897966384 | Val_acc 0.3941


100%|██████████| 40/40 [00:00<00:00, 94.54it/s]


Эпоха: 1 | Train Loss 0.0885946120214007 | Val Loss 2.949818134307861 | Val_acc 0.2565


100%|██████████| 40/40 [00:00<00:00, 100.19it/s]


...

Эпоха: 27 | Train Loss 0.003129715304236078 | Val Loss 3.8403994858264925 | Val_acc 0.2694


100%|██████████| 40/40 [00:00<00:00, 112.43it/s]


Эпоха: 28 | Train Loss 0.00340777502441982 | Val Loss 3.1732634782791136 | Val_acc 0.3346


100%|██████████| 40/40 [00:00<00:00, 126.99it/s]


Эпоха: 29 | Train Loss 0.0021530247024353663 | Val Loss 3.4205375730991356 | Val_acc 0.304


>**Важно.** Не забудь сохранить веса лучшей модели на валидации!

Получилось ли у тебя улучшить результат? Что можно сказать про стабильность `loss` на валидации в сравнении с прошлыми моделями?


нет, слишком сильно нормализует

## Задание 4. Добавление шедулера [2 бонусных балла]

Шедулеры разбирались на неделе, посвящённой оптимизации. Так как нейросети решают оптимизационные задачи, то для них также применяются шедулеры.

В этом задании нужно:
1. Добавить шедулер в код обучения. **[1 бонусный балл]**
2. Выбрать шедулер и обучить с ним модель. **[0,5 бонусного балла]**
3. Описать полученные результаты. **[0,5 бонусного балла]**

Когда можно менять lr шедулером?
* На каждом шаге оптимизатора (в этой задаче сделай так).
* На каждой эпохе.

> **Примечание.** При решении задачи в реальной жизни стоит попробовать оба варианта. Трудно заранее сказать, что лучше сработает для отдельно взятой задачи.

### Твоё решение задания 4

In [ ]:
# Функция для одного шага обучения. Подумай, куда добавить шедулер
def train_step(batch, model, loss, optimizer, device, scheduler=None):

    X, y = batch
    X = X.to(device)
    y = y.to(device)

    logits = model(X)
    l = loss(logits, y)

    l.backward()

    optimizer.step()
    if scheduler:
        scheduler.step()
    model.zero_grad()

    return l.item()

In [ ]:
# Функция для обучения на эпохе. Тут уже аргумент scheduler прокидывается в train_step. То есть ничего тут писать не надо
def train(model, loss, optimizer, device, train_dataloader, scheduler=None):
    model.train()
    train_loss = 0

    for batch in tqdm(train_dataloader):
      loss_step = train_step(batch, model, loss, optimizer, device, scheduler)
      train_loss += loss_step / len(train_dataloader)
    return train_loss

In [ ]:
def train_and_validate(
    epochs,
    model,
    loss,
    optimizer,
    device,
    train_dataloader,
    val_dataloader,
    save_every=1,
    naming="",
    scheduler = None
):

    model.to(device)
    best_acc = -1
    for e in range(epochs):

        train_loss = train(model, loss, optimizer, device, train_dataloader, scheduler=scheduler)
        val_preds, val_loss = validate(model, loss, device, val_dataloader)

        val_targets = torch.cat([y.to(device) for _, y in val_dataloader])
        valid_acc = (val_preds == val_targets.cpu()).numpy().mean()

        print(
            f"Эпоха: {e} | Train Loss {train_loss} | Val Loss {val_loss} | Val_acc {valid_acc}"
        )  # Добавь в print полученное значение с соответствующей подписью 
        if e % save_every == 0 and valid_acc > best_acc:
            torch.save(
                model.state_dict(), f"../6/model_epoch_{e}{naming}{valid_acc}.pth"
            )
            best_acc = valid_acc

Теперь тебе предстоит выбрать шедулер и его параметры. Как с ними быть?

Обычно шедулер и его параметры выбирают такими, чтобы lr по мере обучения только снижался. Это помогает спуститься ещё глубже в минимум, что помогает качественно решить задачу.

В PyTorch уже реализованы самые популярные шедулеры, например: [CosineAnnealingLR](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html) (его идея объяснялась в ДЗ по оптимизации, но сейчас нужен только один цикл, чтобы lr снижался) или [LinearLR](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LinearLR.html).

Можешь выбрать любой другой [шедулер](https://pytorch.org/docs/stable/optim.html#).

In [ ]:
# Инициализируем модель
model = NormFCMNIST()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
loss = nn.CrossEntropyLoss() # Кросс-энтропия, самая популярная функция потерь для решения задачи классификации. Разбиралась на лекциях
epochs = 30
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.01)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Запусти обучение
train_and_validate(epochs, model, loss, optimizer, device, train_dl, test_dl, naming='_sched', scheduler=scheduler) # Нейминг опять другой

100%|██████████| 40/40 [00:00<00:00, 107.41it/s]


Эпоха: 0 | Train Loss 0.6685598221634346 | Val Loss 1.653244712948799 | Val_acc 0.5414


100%|██████████| 40/40 [00:00<00:00, 104.77it/s]


Эпоха: 1 | Train Loss 0.6466911805350345 | Val Loss 1.6401594996452333 | Val_acc 0.5758


100%|██████████| 40/40 [00:00<00:00, 95.94it/s]


...

Эпоха: 27 | Train Loss 0.6477318499614998 | Val Loss 1.6522908180952074 | Val_acc 0.5314


100%|██████████| 40/40 [00:00<00:00, 108.57it/s]


Эпоха: 28 | Train Loss 0.6494006668172756 | Val Loss 1.6605851680040358 | Val_acc 0.5327


100%|██████████| 40/40 [00:00<00:00, 92.52it/s]


Эпоха: 29 | Train Loss 0.6492130986336332 | Val Loss 1.6563156634569172 | Val_acc 0.5451


> **Важно.** Не забудь сохранить веса лучшей модели!

Полчилось ли улучшить качество? Почему такой результат? Что можешь сказать про переобучение?

В целом на мой взгляд получается, что мы слишком сильно регуляризируем из за чего нормально мы ничего не оптимизируем. Шедулер особо погоды не делает т.к мы не доходим до глобального минимума.